In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
# Define functional model

inputs = keras.Input(shape = (28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation = 'relu')

dense2 = keras.layers.Dense(10, activation = 'softmax', name = 'category_output')
dense3 = keras.layers.Dense(1, activation = 'sigmoid',name = 'leftright_output')

In [5]:
x = flatten(inputs)
x = dense1(x)
outputs1 = dense2(x)
outputs2 = dense3(x)

In [6]:
model = keras.Model(inputs = inputs, outputs = [outputs1,outputs2], name = 'mnist_model')

In [7]:
model.summary()

Model: "mnist_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten (Flatten)           (None, 784)                  0         ['input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 128)                  100480    ['flatten[1][0]']             
                                                                                                  
 category_output (Dense)     (None, 10)                   1290      ['dense[0][0]']               
                                                                                        

In [8]:
# loss and optimizer
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
loss2 = keras.losses.BinaryCrossentropy(from_logits = True)
optim = keras.optimizers.Adam(learning_rate = 0.001)
metrics = ['accuracy']

losses = {
    "category_output ": loss1,
    "leftright_output ": loss2
}

model.compile(loss = losses, optimizer = optim, metrics = metrics)

In [17]:
from tensorflow import keras
import numpy as np

# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

# Create a binary label for left (0) and right (1)
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)
y_leftright[y_train > 5] = 1

# Define a model with two output layers
input_layer = keras.layers.Input(shape=(28, 28))
flatten_layer = keras.layers.Flatten()(input_layer)
dense_layer = keras.layers.Dense(128, activation='relu')(flatten_layer)
category_output = keras.layers.Dense(10, activation='softmax', name='category_output')(dense_layer)
leftright_output = keras.layers.Dense(1, activation='sigmoid', name='leftright_output')(dense_layer)

model = keras.models.Model(inputs=input_layer, outputs=[category_output, leftright_output])

# Compile the model with appropriate loss functions and metrics
model.compile(optimizer='adam',
              loss={'category_output': 'sparse_categorical_crossentropy', 'leftright_output': 'binary_crossentropy'},
              metrics={'category_output': ['accuracy'], 'leftright_output': ['accuracy']})

# Train the model
model.fit(X_train, {'category_output': y_train, 'leftright_output': y_leftright}, epochs=5, batch_size=64, verbose=2)


Epoch 1/5
938/938 - 5s - loss: 0.5060 - category_output_loss: 0.3105 - leftright_output_loss: 0.1955 - category_output_accuracy: 0.9137 - leftright_output_accuracy: 0.9271 - 5s/epoch - 5ms/step
Epoch 2/5
938/938 - 4s - loss: 0.2388 - category_output_loss: 0.1401 - leftright_output_loss: 0.0987 - category_output_accuracy: 0.9592 - leftright_output_accuracy: 0.9665 - 4s/epoch - 4ms/step
Epoch 3/5
938/938 - 4s - loss: 0.1772 - category_output_loss: 0.1002 - leftright_output_loss: 0.0769 - category_output_accuracy: 0.9709 - leftright_output_accuracy: 0.9742 - 4s/epoch - 5ms/step
Epoch 4/5
938/938 - 5s - loss: 0.1427 - category_output_loss: 0.0781 - leftright_output_loss: 0.0646 - category_output_accuracy: 0.9772 - leftright_output_accuracy: 0.9782 - 5s/epoch - 5ms/step
Epoch 5/5
938/938 - 4s - loss: 0.1180 - category_output_loss: 0.0630 - leftright_output_loss: 0.0551 - category_output_accuracy: 0.9813 - leftright_output_accuracy: 0.9814 - 4s/epoch - 4ms/step


In [18]:
# predictions
predictions = model.predict(X_test)
len(predictions)

313/313 [==============================] - 1s 2ms/step


2

In [22]:
prediction_category = predictions[0]
predictions_lr = predictions[1]

pr_cat = prediction_category[0:20]
prediction_lr = predictions_lr[0:20]

label_cat = np.argmax(pr_cat, axis = 1)
labels_lr = np.array([1 if p >= 0.5 else 0 for p in predictions_lr])

In [23]:
print(y_test[0:20])
print(label_cat)
print(labels_lr)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 ... 0 0 1]
